## Primeros pasos: hola mundo

correremos un contenedor llamado *hello-world*

    sudo docker run hello-world

En un primer momento no es posible localizarlo localmente, asi que docker intentara encontrarla en el repositorio publico de imagenes, llamado *docker hub*

![](https://i.imgur.com/C0Oz138.png)

Excelente, acabaste de correr tu primer contenedor de docker, el cual se ejecuto, y luego termino. 


## Conceptos fundamentales de Docker: contenedores

El Contenedor es el concepto mas importante. COmenzemos por recordar las VMs, un contenedor podria ser una VM liviana.

Una VM es un programa que corre por ejemplo en nuestra maquina, pero tiene su propia memoria, su propio SO.

Un Contenedor en cambio es una agrupacion de procesos, que corren nativamente en la maquina pero estan aislados del resto del sistema. Un contenedor es una agrupacion logica. 

## Comprendiendo el estado de Docker

Para ver que contenedores estan corriendo, le pedimos al daemon que nos lo muestre, entonces por ejemplo, la primera parte de este comando, la palabra *docker* es el cliente:

    docker ps

![](https://i.imgur.com/leVna9e.png)

Porque ahora estaba en el curso de MongoDB. Este comando solo muestra los contenedores activos. 

Si queremos que nos muestre todos los contenedores, a ver si vemos algo interesante

![](https://i.imgur.com/jMYB4Qy.png)

Entonces cada vez que hacemos *docker run* estamos corriendo un nuevo contenedor, no utilizando una que ya habia, para el caso de *hello world*.

Si queremos mas informacion sobre un contenedor en particular:

    sudo docker inspect 30a692f6c1c8 

Y nos va a dar un huevo de informacion, que por ahora no es relevante. 

 ¿QUe es esa columna de names?

De igual forma sirve para referirse al contenedor de manera amigable, y los names los asigna automaticamente.

    sudo docker inspect infallible_hellman 

Si ahora por ejemplo, queremos crear un nuevo contenedor, pero esta vez asignandole un nuevo nombre, como *hello-platzi*

    sudo docker run --name hello-platzi hello-world

Y volvemos a mostrar todo. 

![](https://i.imgur.com/JjSfitR.png)




Observamos que la imagen es la misma, pero el nombre es ahora el que le asignamos.

Si volvemos a correr el comando anterior, nos dara un error, por lo que tambien es buena idea saber como renombrar los contenedores:

    "docker: Error response from daemon: Conflict. The container name "/hello-platzi" is already in use by container "e74063c777870a10c5d62e6ae8b3e3b23c2201d6c799094a99649d27ab29def4". You have to remove (or rename) that container to be able to reuse that name"

A medida que vamos trabajando se van llenando de mucha informacion, por lo que podemos borrar los contenedores. Usaremos el nombre para removerlos.

    sudo docker rm hello-platzi

Para borrar todos los contenedores parados:

    sudo docker container prune 


## El modo interactivo

Corre un Linux Ubuntu entero dentro de tu PC y de una manera muy facil

    docker run ubuntu  

![](https://i.imgur.com/JEAgPhq.png)

Y aparentemente no paso nada... pero

    sudo docker ps -a

![](https://i.imgur.com/5CafSuk.png)



Observamos que el *STATUS* es apagado, *exited*. ¿Porque? Mira la columna de *COMMAND*, este es el proceso que corre el contenedor al arrancar. Ahora volvamos a mirar la columna *STATUS*, en Linux todo proceso al terminar devuelve un codigo de salida, en este caso un 0, que significa todo sale bien, y si es distinto de cero, significa hubo un problema.

Ahora el contenedor esta apagado, porque el comando */bin/bash* termino. Entonces sabemos que es un Shell, la shell bash, pero como no tiene ningun comando, arranca y como no tiene nada que hacer, pues se apaga.

Volvemos a correr el comando, pero en modo iteractivo(it), 

    sudo docker run -it ubuntu

Y, ahora si cambia el prompt 😀

    root@5bd581a79e8c:/#

¿Pero que es todo esto? Veamos la distribucion de Linux que estamos usando:

    cat etc/lsb-release

Y el resultado es que es una distribucion de Ubuntu 22.04.2

    DISTRIB_ID=Ubuntu
    DISTRIB_RELEASE=22.04
    DISTRIB_CODENAME=jammy
    DISTRIB_DESCRIPTION="Ubuntu 22.04.2 LTS"

Si volvemos a correr en otra terminal el comando *docker ps*

    CONTAINER ID   IMAGE     COMMAND       CREATED         STATUS         PORTS     NAMES
    5bd581a79e8c   ubuntu    "/bin/bash"   7 minutes ago   Up 7 minutes             wizardly_babbage

Ahora si me muestra esta activo





### Ejercicio 

Correr una distribucion de Linux en tu maquina, puede ser *Debian*

    sudo docker run debian
    
Efectivamente, descargo la imagen, y no hizo nada como se esperaba. Sigue:

    sudo docker ps -a

![](https://i.imgur.com/HxPyz0Y.png)

Efectivamente hay un container asociado a la distribucion *Debian*. Y ahora, ponerlo en modo iteractivo:

    sudo docker run -it debian

Y verificando:

    cat etc/issue

Debian GNU/Linux 11 \n \l




## Ciclo de vida de un contenedor

Si volvemos a ejecutar el comando, *docker ps -a*, observamos por ejemplo el contenedor *wizardly baggage* que contine una imagen de Ubuntu, sali hace 8 minutos. Cuando estabamos ejecutando dicho contenedor, se ejecuto el comando *bin/bash*, que es el proceso principal del contenedor

Lo que hay que entender es que cada vez que se ejecuta un contenedor se ejecuta un proceso del SO. Entonces un contedor corre por defecto mientras su proceso principal este corriendo. 

Al hacer *exit* apagamos el proceso principal, y el contenedor se apago. 

Es importante entender porque cuando el *proceso principal* asociado a ese contenedor se detiene o falla, ese contenedor tambien se va a detener. 

### ¿Que pasaria si quiero correr un contenedor de Ubuntu pero que no se apague automaticamente?

ya habiamos visto esto era posible usando el modo iteractivo, pero tal vez esto no es exactamente lo que buscamos.

Entonces tal y como lo hizimos en la seccion de *Comprendiendo el estado de docker* donde creamos un nuevo contenedor llamado *hello-platzi* a partir de la imagen de *hello-world*

    sudo docker run --name hello-platzi hello-world

Vamos hacer algo similar, agregando la opcion *--detach* o *-d* segun tu preferencia, y le agregaremos el comando que queremos corra, en este caso *tail -f /dev/null*

    sudo docker run --name alwaysup --detach ubuntu tail -f /dev/null 

![](https://i.imgur.com/x06l3A0.png)

    



Esa cadena que retorna despues del comando, es el *id* del contenedor, y observamos que efectivamente esta corriendo.


*tail -f /dev/null* es uan aperacion que no hace nada, pero igualmente se queda prendida. Observa la columna COMMAND y veras es el mismo proceso.


### Comando EXEC

Si efectivamente el proceso esta corriendo, pero no puedo hacer nada con el, necesitariamos conectarnos para hacer algo, ¿y como? con *docker exec*

*exec* nos permite en un contenedor que esta corriendo, ejecutar un comando o proceso. A continuacion correremos en modo iteractivo un *shell* (bash)

    sudo docker exec -it alwaysup bash

E inmediatamnte vemos que aparece el *shell*

    root@c862e3060c25:/#

Lo que hay que entender es que este no es el proceso principal. Si ejecutamos el comando *ps* dentro del shell, observamos dos procesos el *ps* como es natural, y el *bash*

![](https://i.imgur.com/QdbnRXD.png)

Si volvemos a ejecutar *ps* con la opcion *aux* veremos todo lo que esta ocurriendo dentro del contenedor. Y efectivamente, ya obsevamos el proceso principal

![](https://i.imgur.com/YtN5q3c.png)

Observa la columna PID(process ID) donde el proces principal es el 1

Si nosotros ejecutamos un *exit* y salimos del *bash*, el contenedor seguira corriendo, porque el proceso principal sigue corriendo.

### Como se detiene o mata el proceso principal.

Y de esta forma detener el contenedor. ¿Cual es el process ID del *tail -f* en mi maquina?

Inspeccionando el contenedor, pero para que no nos muestre todo ese chorrero de informacion, filtramos:

    sudo docker inspect --format '{{.State.Pid}}' alwaysup

Y nos retorna un numero de proceso: 18018

Le damos:

    sudo kill -9 18018



Tambien puedes matar directamente el contenedor con este comando

    sudo docker kill alwaysup 

Y ahora si 😀. Pruebalo si quieres en Docker Lab. 

La diferencia entre docker kill y kill, es que con el primero se mata el contenedor, y con el segundo se mata el proceso principal, para demostra que parando este ultimo, se detiene el contenedor. 



## Exponiendo un COntenedor